## Read input data and turn it into data frame

In [1]:
# Dependancies
import csv
import os
import pandas as pd 

In [2]:
# Input file
input_file = os.path.join("..", "XLY.csv")

# Input file into dataframe
rawdata = pd.read_csv(input_file) 
# Date into date type, date of week, week of year, Gain
rawdata.Date = rawdata.Date.astype('datetime64[ns]')
# isocalendar reports end of the year as 1st week - see 12/31/2008
#rawdata['YearWeek'] = [ td.year + td.isocalendar()[1] / 100 for td in rawdata.Date] 
rawdata['YearWeek'] = [ td.year + (int(td.strftime("%W")) + 1) / 100 for td in rawdata.Date]
rawdata['Weekday'] = [ td.weekday() for td in rawdata.Date]
rawdata['GainPct'] = (rawdata.Close - rawdata.Open) / rawdata.Open
rawdata.tail(10)

,Date,Open,High,Low,Close,Adj Close,Volume,YearWeek,Weekday,GainPct
4907,2019-07-08,121.320000,121.949997,121.199997,121.849998,121.849998,5074600,2019.28,0,0.004369
4908,2019-07-09,121.379997,122.139999,121.250000,121.980003,121.980003,1933300,2019.28,1,0.004943
4909,2019-07-10,122.510002,122.599998,121.769997,122.419998,122.419998,2385000,2019.28,2,-0.000735
4910,2019-07-11,122.830002,123.059998,122.400002,122.669998,122.669998,1976200,2019.28,3,-0.001303
4911,2019-07-12,123.010002,124.110001,123.010002,124.080002,124.080002,2487600,2019.28,4,0.008698
4912,2019-07-15,124.349998,124.540001,123.849998,124.480003,124.480003,1996600,2019.29,0,0.001045
4913,2019-07-16,124.300003,124.599998,123.930000,124.320000,124.320000,2051700,2019.29,1,0.000161
4914,2019-07-17,124.169998,124.349998,123.129997,123.129997,123.129997,2299600,2019.29,2,-0.008376
4915,2019-07-18,122.879997,123.209999,122.150002,122.970001,122.970001,2323100,2019.29,3,0.000732
4916,2019-07-19,123.349998,123.519997,122.089996,122.160004,122.160004,2064100,2019.29,4,-0.009647


In [3]:
# Pivot data into new df
#rawdata['ddd'] = rawdata.duplicated(subset=['YearWeek','Weekday'], keep=False )
#rawdata[rawdata.ddd]
middata = rawdata.pivot(index='YearWeek', columns='Weekday', values='GainPct')
middata.head()

Weekday,0,1,2,3,4
YearWeek,,,,,
2000.02,-0.028226,-0.015789,-0.016507,0.004841,0.036017
2000.03,-0.004094,-0.008709,-0.008247,0.000000,-0.003616
2000.04,NaN,-0.001042,-0.001565,-0.018711,-0.021693
2000.05,-0.044039,0.002800,0.016911,-0.010451,-0.023729
2000.06,0.012209,0.027335,-0.008850,-0.006077,-0.010469


In [4]:
# Add column with maximum gain of the week
middata['MaxGainDay'] = middata.idxmax(axis=1)
middata.head()

Weekday,0,1,2,3,4,MaxGainDay
YearWeek,,,,,,
2000.02,-0.028226,-0.015789,-0.016507,0.004841,0.036017,4
2000.03,-0.004094,-0.008709,-0.008247,0.000000,-0.003616,3
2000.04,NaN,-0.001042,-0.001565,-0.018711,-0.021693,1
2000.05,-0.044039,0.002800,0.016911,-0.010451,-0.023729,2
2000.06,0.012209,0.027335,-0.008850,-0.006077,-0.010469,1


In [5]:
# Get basic aggregation
middata.groupby('MaxGainDay').size()

MaxGainDay
0    183
1    238
2    204
3    240
4    164
dtype: int64